<a href="https://colab.research.google.com/github/gunisha30/codes/blob/master/Copy_of_audio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install praat-parselmouth

In [0]:
import sys
import os
from moviepy.editor import *

# name is the path, if its in the root directory then its just the video name
def convert_to_audio(name):
  video = VideoFileClip(name)
  audio = video.audio
  audio.write_audiofile('Q6converted.wav')

os.chdir('/content/drive/My Drive/awarathon/third')
name="Q6.mp4"
convert_to_audio(name)

[MoviePy] Writing audio in Q6converted.wav


100%|██████████| 199/199 [00:00<00:00, 1876.57it/s]

[MoviePy] Done.


In [0]:
import parselmouth
from parselmouth.praat import call, run_file
import inspect as i
import sys
import scipy.io.wavfile

def audiolength(audio,path): #in mins
    rate, arr = scipy.io.wavfile.read(path+"/"+audio+".wav")
    length=arr.size/rate
    length=length/60
    return(length)

def mysppaus(m,p,l): #no of pauses
    sound=p+"/"+m+".wav" #entire path till audio file
    sourcerun="/content/drive/My Drive/awarathon/myspsolution.praat" #path till mysolution.praat
    path=p #path till the folder which has audio file
    try:
        objects= run_file(sourcerun, -20, 2, 1, "yes",sound,path, 80, 400, 0.01, capture_output=True)
        z1=str(objects[1])
        z2=z1.strip().split()
        print ("articulation rate=",z2[3])
        print (z2)
        p=int(z2[1]) #pause length in secs
        p=p/60 #pause length in mins
        a=p/l #ratio of pause length and total length
        print("rate of pause=",a)
        print ("pause percentage=",a*100)
    except:
        print ("Try again the sound of the audio was not clear")
    return;


audioname="Q6converted"
path1="/content/drive/My Drive/awarathon/third"
l=audiolength(audioname,path1)
print(l)
mysppaus(audioname,path1,l)

0.3
articulation rate= 4
['28', '0', '3', '4', '6.4', '9', '0.7', '179.49', '32.31', '174.4', '75', '272', '157', '195', '0.9']
rate of pause= 0.0
pause percentage= 0.0


In [0]:
from google.colab import drive
drive.mount('/content/drive')